In [ ]:
# Importing necessary libraries for machine learning and deep learning
import transformers  # Hugging Face library for working with pre-trained models
import torch        # PyTorch library for tensor computations and neural networks
import typing       # Library for type hinting and type annotations





In [ ]:
class MedicalChatbot:
    # Dictionary mapping medical question categories to tailored system prompts
    category_prompts = {
        "Basic Medical Knowledge": (
            "You are an experienced medical professional specializing in general medical knowledge. "
            "Provide detailed and comprehensible answers to basic medical questions, ensuring accuracy and clarity."
        ),
        "Pharmacological Queries": (
            "You are a pharmacology expert with extensive knowledge of drugs, their mechanisms, and side effects. "
            "Provide precise and evidence-based information about medications and treatments."
        ),
        "Diagnostic Reasoning": (
            "You are a diagnostic expert proficient in identifying medical conditions. "
            "Offer clear and methodical guidance for diagnostic reasoning and differential diagnoses."
        ),
        "Treatment & Management": (
            "You are a specialist in medical treatment and patient management strategies. "
            "Provide comprehensive explanations of treatment guidelines and management protocols."
        ),
        "Specialized Medical Topics": (
            "You are an expert in advanced medical research and innovations. "
            "Discuss specialized medical topics with a focus on the latest scientific advances."
        ),
        "Preventive Medicine": (
            "You are a preventive medicine specialist with expertise in lifestyle modifications and public health. "
            "Explain preventive strategies and their importance in reducing disease risk."
        ),
        "Specialized Medical Scenarios": (
            "You are a clinical specialist adept at handling unique and complex medical scenarios. "
            "Provide nuanced insights into complications and specialized conditions."
        )
    }

    # Fallback system prompt for undefined categories
    default_prompt = (
        "You are an expert and experienced medical professional with extensive medical knowledge. "
        "Provide precise, evidence-based medical explanations that are scientifically accurate and comprehensible to a general audience."
    )

    def __init__(self, model_id="aaditya/OpenBioLLM-Llama3-70B"):
        """
        Initialize the medical chatbot with a specific medical language model
        
        Args:
            model_id (str): Identifier for the pre-trained medical language model
        """

        try:
            # Import logging for better error tracking and debugging
            import logging

            # Configure logging
            logging.basicConfig(level=logging.INFO, 
                                format='%(asctime)s - %(levelname)s - %(message)s')
            self.logger = logging.getLogger(__name__)

            # Log model initialization
            self.logger.info(f"Initializing Medical Chatbot with model: {model_id}")

            # Create a text generation pipeline using the specified model
            self.pipeline = transformers.pipeline(
                "text-generation",            # Specify the task as text generation
                model=model_id,               # Use the specified model from Hugging Face
                model_kwargs={"torch_dtype": torch.bfloat16},  # Use lower precision for memory efficiency
                device="auto",                # Automatically select best available device (CPU/GPU)
            )
        # Log successful model loading
            self.logger.info("Model successfully initialized")

        except Exception as e:
            # More robust error handling
            self.logger.error(f"Failed to initialize model: {e}")
            raise RuntimeError(f"Model initialization failed: {e}")
        

        # Default medical category (can be dynamically updated later)
        self.med_cat = "Basic Medical Knowledge"

        # Fetch the initial system prompt based on the category
        self.system_prompt = self.category_prompts.get(self.med_cat, self.default_prompt)

    def set_category(self, category: str):
        """
        Set the medical category dynamically and update the system prompt.
        
        Args:
            category (str): New medical category to set.
        """
        self.med_cat = category
        self.system_prompt = self.category_prompts.get(self.med_cat, self.default_prompt)

    def generate_response(
        self, 
        user_query: str,           # Type hint for user's input query
        max_tokens: int = 256,     # Default maximum response length
        temperature: float = 0.1,   # Default temperature for response variability
        safety_threshold: float = 0.7  # New parameter for response safety
    ) -> str:                      # Type hint indicating return is a string
        """
        Generate a medical response to a user's query
        
        Args:
            user_query (str): Medical question or prompt
            max_tokens (int): Maximum response length
            temperature (float): Controls response randomness/creativity
        
        Returns:
            str: Generated medical response
        """

        # Input validation
        if not user_query or not isinstance(user_query, str):
            raise ValueError("User query must be a non-empty string")
        
        # Ensure temperature is within a reasonable range
        temperature = max(0.0, min(temperature, 1.0))


        try:
            # Construct message list with system and user roles
            messages = [
                {"role": "system", "content": self.system_prompt},  # System prompt defining AI's role
                {"role": "user", "content": user_query}             # User query for the model
            ]

            # Apply chat template to format messages for the model
            prompt = self.pipeline.tokenizer.apply_chat_template(
                messages,
                tokenize=False,               # Return as string, not tokens
                add_generation_prompt=True    # Add markers for response generation
            )

            # Define token IDs to terminate generation
            terminators = [
                self.pipeline.tokenizer.eos_token_id,                 # Standard end-of-sequence token
                self.pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # Custom end token
            ]

            # Generate response using the language model
            outputs = self.pipeline(
                prompt,
                max_new_tokens=max_tokens,    # Limit response length
                eos_token_id=terminators,     # Use defined termination tokens
                do_sample=True,               # Enable probabilistic sampling
                temperature=temperature,       # Control response randomness
                top_p=0.9,                    # Nucleus sampling parameter
                top_k=50,  # Added top-k sampling for more controlled generation
            )

            # Extract and clean the generated response
            generated_text = outputs[0]["generated_text"][len(prompt):].strip()

            # Optional: Simple safety filtering (very basic, consider more robust solutions)
            if len(generated_text.split()) / max_tokens > safety_threshold:
                self.logger.warning("Generated response might exceed safety threshold")

            return generated_text
        
        except Exception as e:
            # Comprehensive error logging
            self.logger.error(f"Response generation failed: {e}")
            return f"An error occurred while processing your query: {str(e)}"

In [5]:
# List of predefined medical questions for testing the model catogorized with dictionaries 
medical_questions = {
    "Basic Medical Knowledge": [
        "What are the primary symptoms of type 2 diabetes?",
        "Explain the pathophysiology of hypertension.",
        "What are the recommended screening protocols for breast cancer?"
    ],
    "Pharmacological Queries": [
        "What are the potential side effects of statins?",
        "How do ACE inhibitors work to manage blood pressure?",
        "Compare the mechanisms of different antidepressant classes."
    ],
    "Diagnostic Reasoning": [
        "What diagnostic tests would you recommend for suspected rheumatoid arthritis?",
        "Describe the differential diagnosis for chest pain in a 45-year-old male."
    ],
    "Treatment & Management": [
        "What are current guidelines for managing type 1 diabetes in adolescents?",
        "Explain the stages of cancer treatment and potential therapies."
    ],
    "Specialized Medical Topics": [
        "How does CRISPR technology potentially impact genetic disease treatment?",
        "What are the latest advances in immunotherapy for cancer?"
    ],
    "Preventive Medicine": [
        "What lifestyle modifications can reduce the risk of cardiovascular disease?",
        "Discuss the importance of vaccination in preventing infectious diseases."
    ],
    "Specialized Medical Scenarios": [
        "What are the complications of untreated gestational diabetes?",
        "Explain the neurological manifestations of multiple sclerosis."
    ]
}

med_cat = "Basic Medical Knowledge" #select the medical category for the user query
User_questions = medical_questions[med_cat] #set the users questions to the selected category


In [ ]:
# Main function to demonstrate the chatbot's functionality
def main():
    # Create an instance of the MedicalChatbot
    chatbot = MedicalChatbot()
    
    # Iterate through a subset of medical questions
    for question in User_questions:  # Test the selected questions
        # Print the current query
        print(f"\n🩺 Query: {question}")
        
        # Generate and print the response
        response = chatbot.generate_response(question)
        print(f"📝 Response: {response}")
        
        # Print a separator for readability
        print("-" * 50)



In [ ]:
# Ensure the main function only runs if the script is executed directly
if __name__ == "__main__":
    main()